# Fetch all GitHub Release info for CNCF projects

In [1]:
#!wget https://landscape.cncf.io/data/items.json

In [2]:
%%bash 
# array of JSON --> JSONL
jq  -c '.[]' items.json > items.jsonl

ls -lahF items.jsonl
wc -l items.jsonl

-rw-r--r--  1 me  staff   9.7M Nov 14 11:02 items.jsonl
    2267 items.jsonl


In [3]:
%%bash
# landscape | select(CNCF Projects) --> cncf-projects.jsonl
jq -c 'select(.relation == "graduated" or .relation == "incubating" or .relation == "sandbox")' items.jsonl > cncf-projects.jsonl 

ls -lahF cncf-projects.jsonl
wc -l cncf-projects.jsonl

-rw-r--r--  1 me  staff   3.9M Nov 14 11:02 cncf-projects.jsonl
     178 cncf-projects.jsonl


## helpers

In [4]:
import pandas as pd

pd.set_option('display.max_rows', 512)
pd.set_option('display.max_columns', 512)
pd.set_option('display.width', 512)

# for PAT / token
from dotenv import load_dotenv
load_dotenv()

def safe_set_index(df:         pd.DataFrame, 
                   idx_wanted: list[str]) -> pd.DataFrame:

    # check to see if the index is already set, else, data loss as set_index can be destructive
    idx_existing = list(df.index.names)

    if idx_wanted == idx_existing:
        print(f'\n*** WARNING: attempt to set index to what it already is thwarted! \n')
    else:
        if idx_existing:
            print(f'*** existing index found: {idx_existing}, resetting to avoid data loss prior to setting to: {idx_wanted}')
            df.reset_index(inplace=True)

        # note: index must be unique!
        df.set_index(idx_wanted, verify_integrity=True, inplace=True)
    
    df.sort_index(inplace=True)
    return df

def split_org_repo(df:      pd.DataFrame, 
                   colname: str,
                   drop:    bool = False,
                   newcol_org_name:  str = 'org_name',
                   newcol_repo_name: str = 'repo_name') -> pd.DataFrame:
    '''split_org_repo(df, colname) - org_name/repo_name --> org_name, repo_named'''
    
    if colname is None:
        raise ValueError('split_org_repo: missing colname!')

    # expand=True returns df which can rename columns on # https://swdevnotes.com/python/2022/extract-data-from-json-in-pandas-dataframe
    
    df_newcols = df[colname].copy().str.split(pat='/', n=1, expand=True)
    df_newcols.rename(columns={0: newcol_org_name, 1: newcol_repo_name}, inplace=True)

    if drop:
        df.drop(colname, axis=1, inplace=True)

    df = pd.concat([df,df_newcols], axis=1)
    return df

## Load the Landscape 

In [5]:
import os

file_path = './cncf-projects.jsonl'

if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    df = pd.read_json(file_path, lines=True)
else:
    print(f"File {file_path} does not exist or is empty.")

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 55 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      178 non-null    object 
 1   homepage_url              178 non-null    object 
 2   project                   178 non-null    object 
 3   repo_url                  176 non-null    object 
 4   logo                      178 non-null    object 
 5   twitter                   174 non-null    object 
 6   crunchbase                178 non-null    object 
 7   extra                     176 non-null    object 
 8   github_data               176 non-null    object 
 9   repos                     176 non-null    object 
 10  github_start_commit_data  176 non-null    object 
 11  image_data                178 non-null    object 
 12  firstCommitDate           176 non-null    object 
 13  firstCommitLink           176 non-null    object 
 14  latestComm

In [6]:
df.head(5)

,name,homepage_url,project,repo_url,logo,twitter,crunchbase,extra,github_data,repos,github_start_commit_data,image_data,firstCommitDate,firstCommitLink,latestCommitDate,latestCommitLink,releaseDate,releaseLink,commitsThisYear,contributorsCount,contributorsLink,language,stars,license,headquarters,latestTweetDate,description,organization,crunchbaseData,path,landscape,category,amountKind,amount,oss,href,bestPracticeBadgeId,bestPracticePercentage,industries,starsPresent,starsAsText,marketCapPresent,marketCapAsText,id,flatName,member,relation,isSubsidiaryProject,allow_duplicate_repo,project_org,joined,enduser,url_for_bestpractices,open_source,second_path
0,Akri,https://docs.akri.sh,sandbox,https://github.com/project-akri/akri,akri.svg,https://twitter.com/ProjectAkri,https://www.crunchbase.com/organization/cloud-...,"{'accepted': '2021-09-14', 'annual_review_date...","{'languages': [{'name': 'Rust', 'value': 10535...",[{'url': 'https://github.com/project-akri/akri...,{'start_commit_link': '/project-akri/akri/comm...,"{'fileName': 'akri.svg', 'hash': '2nK42JQaM8qF...",2020-10-14T00:42:19Z,https://github.com/project-akri/akri/commit/94...,"{'text': '2 months ago', 'value': '950', 'orig...",https://github.com/project-akri/akri/commit/98...,"{'text': '7 months ago', 'value': '900', 'orig...",https://github.com/project-akri/akri/releases,197,34.0,https://github.com/project-akri/akri/graphs/co...,Rust,996.0,Apache License 2.0,"San Francisco, California","{'text': '7 months ago', 'value': '900', 'orig...",A Kubernetes Resource Interface for the Edge,Cloud Native Computing Foundation (CNCF),{'name': 'Cloud Native Computing Foundation (C...,Provisioning / Automation & Configuration,Provisioning / Automation & Configuration,Provisioning,funding,3000000,True,logos/akri.svg,5339,99.0,"[Cloud Computing, Cloud Infrastructure, Non Pr...",True,996,True,$3M,akri,Akri,False,sandbox,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CDK for Kubernetes (CDK8s),https://cdk8s.io/,sandbox,https://github.com/cdk8s-team/cdk8s,cdk8s.svg,https://twitter.com/CloudNativeFdn,https://www.crunchbase.com/organization/cloud-...,"{'accepted': '2020-11-10', 'annual_review_url'...","{'languages': [{'name': 'JavaScript', 'value':...",[{'url': 'https://github.com/cdk8s-team/cdk8s'...,{'start_commit_link': '/cdk8s-team/cdk8s/commi...,"{'fileName': 'cdk-for-kubernetes-cdk8s.svg', '...",2019-07-25T12:45:22Z,https://github.com/cdk8s-team/cdk8s/commit/ec7...,"{'text': 'about a month', 'value': '960', 'ori...",https://github.com/cdk8s-team/cdk8s/commit/6b9...,2021-10-13T20:21:32Z,https://github.com/cdk8s-team/cdk8s/releases,434,78.0,https://github.com/cdk8s-team/cdk8s/graphs/con...,JavaScript,3818.0,Apache License 2.0,"San Francisco, California","{'text': '7 months ago', 'value': '900', 'orig...",CDK8s lets you define Kubernetes apps and comp...,Cloud Native Computing Foundation (CNCF),{'name': 'Cloud Native Computing Foundation (C...,Provisioning / Automation & Configuration,Provisioning / Automation & Configuration,Provisioning,funding,3000000,True,logos/cdk-for-kubernetes-cdk8s.svg,0,NaN,"[Cloud Computing, Cloud Infrastructure, Non Pr...",True,"3,818",True,$3M,cdk-for-kubernetes-cdk8s,CDK for Kubernetes (CDK8s),False,sandbox,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cloud Custodian,https://cloudcustodian.io/,incubating,https://github.com/cloud-custodian/cloud-custo...,cloud-custodian.svg,https://twitter.com/CloudNativeFdn,https://www.crunchbase.com/organization/cloud-...,"{'accepted': '2020-06-25', 'dev_stats_url': 'h...","{'languages': [{'name': 'Python', 'value': 771...",[{'url': 'https://github.com/cloud-custodian/c...,{'start_commit_link': '/cloud-custodian/cloud-...,"{'fileName': 'cloud-custodian.svg', 'hash': '0...",2015-07-16T14:19:15Z,https://github.com/cloud-custodian/cloud-custo...,"{'text': '2 months ago', 'value': '950', 'orig...",https://github.com/cloud-custodian/cloud-custo...,"{'text': '2 months ago', 'value': '950', 'orig...",https://github.com/cloud-custodian/cloud-custo...,52

In [7]:
# extract a clean org/repo
df['repo'] = df['repo_url'].astype('string').str.removeprefix('https://github.com/').dropna()
df = split_org_repo(df, 'repo')
df.head(1)

,name,homepage_url,project,repo_url,logo,twitter,crunchbase,extra,github_data,repos,github_start_commit_data,image_data,firstCommitDate,firstCommitLink,latestCommitDate,latestCommitLink,releaseDate,releaseLink,commitsThisYear,contributorsCount,contributorsLink,language,stars,license,headquarters,latestTweetDate,description,organization,crunchbaseData,path,landscape,category,amountKind,amount,oss,href,bestPracticeBadgeId,bestPracticePercentage,industries,starsPresent,starsAsText,marketCapPresent,marketCapAsText,id,flatName,member,relation,isSubsidiaryProject,allow_duplicate_repo,project_org,joined,enduser,url_for_bestpractices,open_source,second_path,repo,org_name,repo_name
0,Akri,https://docs.akri.sh,sandbox,https://github.com/project-akri/akri,akri.svg,https://twitter.com/ProjectAkri,https://www.crunchbase.com/organization/cloud-...,"{'accepted': '2021-09-14', 'annual_review_date...","{'languages': [{'name': 'Rust', 'value': 10535...",[{'url': 'https://github.com/project-akri/akri...,{'start_commit_link': '/project-akri/akri/comm...,"{'fileName': 'akri.svg', 'hash': '2nK42JQaM8qF...",2020-10-14T00:42:19Z,https://github.com/project-akri/akri/commit/94...,"{'text': '2 months ago', 'value': '950', 'orig...",https://github.com/project-akri/akri/commit/98...,"{'text': '7 months ago', 'value': '900', 'orig...",https://github.com/project-akri/akri/releases,197,34.0,https://github.com/project-akri/akri/graphs/co...,Rust,996.0,Apache License 2.0,"San Francisco, California","{'text': '7 months ago', 'value': '900', 'orig...",A Kubernetes Resource Interface for the Edge,Cloud Native Computing Foundation (CNCF),{'name': 'Cloud Native Computing Foundation (C...,Provisioning / Automation & Configuration,Provisioning / Automation & Configuration,Provisioning,funding,3000000,True,logos/akri.svg,5339,99.0,"[Cloud Computing, Cloud Infrastructure, Non Pr...",True,996,True,$3M,akri,Akri,False,sandbox,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,project-akri/akri,project-akri,akri


In [9]:
# REINDEX
df['subpath'] = df['path']
safe_set_index(df, idx_wanted=['relation', 'category', 'path', 'name'])

*** existing index found: ['category', 'relation', 'path', 'name'], resetting to avoid data loss prior to setting to: ['relation', 'category', 'path', 'name']


index                                       homepage_url     project                                           repo_url                                 logo                              twitter                                         crunchbase                                              extra                                        github_data  \
relation   category                       path                                               name                                                                                                                                                                                                                                                                                                                                                                                              
graduated  App Definition and Development App Definition and Development / Application De... Helm                                   124                                   https://helm.sh/   graduated                       https://github.com/helm/helm                             helm.svg         https://twitter.com/helmpack  https://www.crunchbase.com/organization/cloud-...  {'accepted': '2018-06-01', 'incubating': '2018...  {'languages': [{'name': 'Go', 'value': 1807584...   
                                          App Definition and Development / Continuous Int... Argo                                   139                        https://argoproj.github.io/   graduated                https://github.com/argoproj/argo-cd                             argo.svg         https://twitter.com/argoproj  https://www.crunchbase.com/organization/cloud-...  {'accepted': '2020-03-26', 'incubating': '2020...  {'languages': [{'name': 'Go', 'value': 5524117...   
                                                                                             Flux                                   140                                 https://fluxcd.io/   graduated                    https://github.com/fluxcd/flux2                             flux.svg           https://twitter.com/fluxcd  https://www.crunchbase.com/organization/cloud-...  {'accepted': '2019-07-15', 'incubating': '2021...  {'languages': [{'name': 'Go', 'value': 970748,...   
                                          App Definition and Development / Database          TiKV                                   110                                   https://tikv.org   graduated                       https://github.com/tikv/tikv                             tikv.svg      https://twitter.com/tikvproject  https://www.crunchbase.com/organization/cloud-...  {'accepted': '2018-08-28', 'incubating': '2018...  {'languages': [{'name': 'Rust', 'value': 21095...   
                                                                                             Vitess                                 111                                 https://vitess.io/   graduated                 https://github.com/vitessio/vitess                           vitess.svg         https://twitter.com/vitessio  https://www.crunchbase.com/organization/cloud-...  {'accepted': '2018-02-05', 'incubating': '2018...  {'languages': [{'name': 'Go', 'value': 2680572...   
           Observability and Analysis     Observability and Analysis / Logging               Fluentd                                167                           https://www.fluentd.org/   graduated                  https://github.com/fluent/fluentd                          fluentd.svg          https://twitter.com/fluentd  https://www.crunchbase.com/organization/cloud-...  {'accepted': '2016-11-08', 'incubating': '2016...  {'languages': [{'name': 'Ruby', 'value': 32604...   
                                          Observability and Analysis / Monitoring            Prometheus                             163                             https://prometheus.io/   graduated           https://github.com/prometheus/prometheus                       prometheus.svg    

In [ ]:
df.shape
df.info()

In [ ]:
# df2=df[['B','D','F']].rename({'B':'X','D':'Y','F':'Z'}, axis=1)

# TODO there's a lot of duplicative stuff, trim, for now too small to matter
cols_mini = ['relation',
             'org_name', 
             'repo_name', 
             'repo', 
             'name',
             'category', 
             'path', 
             'repo_url',
             'github_data',
             'extra',
             'language',
             'industries',
             'contributorsCount']

df_mini = df.reset_index()[cols_mini]
safe_set_index(df_mini, idx_wanted=['category', 'relation', 'name'])

df_mini.info()
df_mini

## Just the Observability Domain (for now)

In [ ]:
# df_otag = df_mini.query("`category` == 'Observability and Analysis'").copy()

# TODO: make this for all tags
TAG = 'Observability and Analysis'

df_otag = df_mini.loc[TAG, :, :].copy()
df_otag.info()
df_otag

## Generate (OTAG) full repo list

In [ ]:
orgs = df_otag.org_name.drop_duplicates().tolist()
orgs

In [ ]:
# get_repos(type: Opt[str] = NotSet, sort: Opt[str] = NotSet, direction: Opt[str] = NotSet) → PaginatedList[Repository]¶
# Calls
# GET /orgs/{org}/repos

# Parameters
# type – string (‘all’, ‘public’, ‘private’, ‘forks’, ‘sources’, ‘member’)

# sort – string (‘created’, ‘updated’, ‘pushed’, ‘full_name’)

# direction – string (‘asc’, desc’)

for o in orgs:
    # get a list of all public repositories in each org.
    



## Fetch project release data from GitHub API

In [ ]:
import time
import os
import json
import pandas as pd
from typing import List
from datetime import datetime, timezone
from github import Github, GithubException

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

def fetch_repo_data(token: str, 
                    repo_list: List[str], 
                    since: datetime=None, 
                    json_file: str=None, 
                    csv_file: str=None, 
                    state_file: str=None) -> pd.DataFrame:

    # Initialize DataFrame
    # df = pd.DataFrame(columns=[
    #     'repo_name', 'release_name', 'release_date', 
    #     'language', 'release_notes'
    # ])

    df = pd.DataFrame(columns=['repo_name', 'release_name', 'release_date', 'language'])

    # Initialize GitHub client
    g = Github(token)

    # Initialize loop state
    if state_file:
        try:
            with open(state_file, 'r') as f:
                state = json.load(f)
        except FileNotFoundError:
            state = {'i': 0, 'repos_done': []}
    else:
        state = {'i': 0, 'repos_done': []}

    # Loop over repositories
    while state['i'] < len(repo_list):
        repo_str = repo_list[state['i']]

        if repo_str in state['repos_done']:
            print(f"Skipping: {repo_str}")
            state['i'] += 1
            continue

        while True:
            try:
                repo = g.get_repo(repo_str)
                break
            except GithubException as e:
                if e.status == 404:
                    print(f"Repository {repo_str} not found")
                    break
                elif e.status == 429:
                    print(f"Rate limit exceeded, waiting for {e.headers['Retry-After']} seconds...")
                    time.sleep(int(e.headers['Retry-After']))
                else:
                    print(f"Error getting repository {repo_str}: {e}")
                    break

        if not repo:
            state['i'] += 1
            continue

        # Get all releases
        releases = repo.get_releases()
        language = repo.language

        for release in releases:
            if since is None or release.created_at >= since:
                df = pd.concat([df, pd.DataFrame({
                    'repo_name': [repo_str],
                    'release_name': [release.title],
                    'release_date': [str(release.published_at)],
                    'language': [language],
                    #'release_notes': [release.body]
                })])
                print(f"Added {release.published_at}, {repo_str}::{release.title}  ")

        # Save state
        if state_file:
            state['repos_done'].append(repo_str)
            with open(state_file, 'w') as f:
                json.dump(state, f, indent=4)

        state['i'] += 1

    #print (releases)
    
    # Save as CSV
    if csv_file:
        df.to_csv(csv_file, index=False)

    # Save as JSON
    if json_file:
        df.to_json(json_file, orient='records', lines=True)
    return df

In [ ]:
import time
import os
import json
import pandas as pd
from typing import List
from datetime import datetime, timezone
from github import Github, GithubException

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

def fetch_org_data(token: str, 
                   org_list: List[str], 
                   since: datetime=None, 
                   json_file: str=None, 
                   csv_file: str=None, 
                   state_file: str=None) -> pd.DataFrame:

    # Initialize DataFrame
    # df = pd.DataFrame(columns=[
    #     'org_name', 'release_name', 'release_date', 
    #     'language', 'release_notes'
    # ])

    df = pd.DataFrame(columns=['org_name', 'release_name', 'release_date', 'language'])

    # Initialize GitHub client
    g = Github(token)

    # Initialize loop state
    if state_file:
        try:
            with open(state_file, 'r') as f:
                state = json.load(f)
        except FileNotFoundError:
            state = {'i': 0, 'orgs_done': []}
    else:
        state = {'i': 0, 'orgs_done': []}

    # Loop over orgs
    while state['i'] < len(org_list):
        org_str = org_list[state['i']]

        if org_str in state['orgs_done']:
            print(f"Skipping: {org_str}")
            state['i'] += 1
            continue

        while True:
            try:
                org = g.get_org(org_str)
                break
            except GithubException as e:
                if e.status == 404:
                    print(f"Org {org_str} not found")
                    break
                elif e.status == 429:
                    print(f"Rate limit exceeded, waiting for {e.headers['Retry-After']} seconds...")
                    time.sleep(int(e.headers['Retry-After']))
                else:
                    print(f"Error getting org {org_str}: {e}")
                    break

        if not org:
            state['i'] += 1
            continue

        # releases = org.get_releases()
        # language = org.language

        for release in releases:
            if since is None or release.created_at >= since:
                df = pd.concat([df, pd.DataFrame({
                    'org_name': [org_str],
                    'release_name': [release.title],
                    'release_date': [str(release.published_at)],
                    'language': [language],
                    #'release_notes': [release.body]
                })])
                print(f"Added {release.published_at}, {org_str}::{release.title}  ")

        # Save state
        if state_file:
            state[orgs_done'].append(org_str)
            with open(state_file, 'w') as f:
                json.dump(state, f, indent=4)

        state['i'] += 1

    #print (releases)
    
    # Save as CSV
    if csv_file:
        df.to_csv(csv_file, index=False)

    # Save as JSON
    if json_file:
        df.to_json(json_file, orient='records', lines=True)
    return df

In [ ]:
otag_repos_by_relation

In [ ]:
!mkdir -p out

In [ ]:
def fetch_one(token, since_date, level, repos):

    json_file=f'out/cncf-{level}-github-releases.json' 
    csv_file=f'out/cncf-{level}-github-releases.csv'
    state_file=f'out/.nukeme_state_file_{level}'
    
    print(f"Fetching {len(repos)} repositories for {level} projects")
    
    releases = fetch_repo_data( token, 
                                repos, 
                                since=since_date,
                                json_file=json_file,
                                csv_file=csv_file)

In [ ]:
token = os.environ['GITHUB_TOKEN']
since_date = datetime(2022, 11, 7, tzinfo=timezone.utc)

fetch_one(token, since_date, 'otag-graduated', graduated)

In [ ]:
fetch_one(token, since_date, 'otag-incubating', incubating)

In [ ]:
fetch_one(token, since_date, 'otag-sandbox', sandbox)

In [ ]:
%pip install ipympl

%matplotlib inline
%matplotlib widget

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

# def plot_releases_timeline(releases: pd.DataFrame):
#     fig = px.timeline(releases, x_start="release_date", x_end="release_date", y="repo_name", color="language", title="GitHub Releases Timeline")
#     fig.update_yaxes(autorange="reversed")
#     fig.show()

# def plot_releases_scatter_simple(releases: pd.DataFrame):
#     # Filter releases by year
#     releases_2023 = releases[releases['release_date'].dt.year == 2023]

#     # Create scatter plot
#     fig = px.scatter(releases_2023, x="release_date", y="repo_name", color="language")
#     fig.update_yaxes(autorange="reversed")
#     fig.show()

def plot_releases_scatter(releases: pd.DataFrame, title: str=None):
    # # Filter releases by year
    # releases_2023 = releases[releases['release_date'].dt.year == 2023]

    if title is None:
        title = "GitHub Releases Timeline"

    # Group releases by organization
    releases['organization'] = releases['repo_name'].apply(lambda x: x.split('/')[0])

    # Create scatter plot
    fig = px.scatter(releases, x="release_date", y="repo_name", color="organization", symbol="language", title="Project Releases")
    fig.update_yaxes(autorange="reversed")
    
    fig.update_layout(showlegend=True,
                      autosize=True,
                      width=1000)
                    #   height=2500,
                    #   )
    fig.show()

In [ ]:
import pandas as pd
import os

def json_to_csv(json_file_path: str) -> None:
    """
    Load a JSON file into a pandas DataFrame and save it as a CSV file with the same name.
    """
    df = pd.read_json(json_file_path, lines=True)
    
    csv_file_path = os.path.splitext(json_file_path)[0] + '.csv'
    df.to_csv(csv_file_path, index=False)

In [ ]:
for level in ['graduated', 'incubating', 'sandbox']:
    json_to_csv(f'out/cncf-{level}-github-releases.json')

In [ ]:
def plot_releases_from_csv(csv_file: str, title: str) -> None:
    csv_file =f'out/cncf-{level}-github-releases.csv'
    if os.path.exists(csv_file):
        df_releases = pd.read_csv(csv_file)
        df_releases.release_date = pd.to_datetime(df_releases.release_date)
        
        plot_releases_scatter(df_releases, title)
    else:
        print(f"CSV file {csv_file} not found")

In [ ]:
import pandas as pd
import os

for level in ['graduated', 'incubating', 'sandbox']:
    plot_releases_from_csv(f'out/cncf-{level}-github-releases.csv', f'Releases: {level}')


In [ ]:
import pandas as pd
import os


# plot_releases_scatter(f'out/cncf-all-github-releases.csv')



In [ ]:
releases_by_repo = releases[['repo_name', 'release_date']].groupby('repo_name').count()
releases_by_repo.to_csv('cncf_releases_by_repo.csv')